In [141]:
from keras.datasets import mnist

In [142]:
import numpy as np
import pandas as pd
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [143]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [144]:
px.imshow(x_train[16], width=500)

#### Preprocess data for modelling

In [145]:
# convert x values from 0-255 to 0-1
x_train = x_train/255
x_test = x_test/255

# convert y values from integer to binary array
def y_categorical(y):
    res = []
    for i in range(len(y)):
        arr = [0]*10
        arr[y[i]] = 1
        res.append(arr)
    return res
y_train = y_categorical(y_train)
y_test = y_categorical(y_test)

# convert numpy to pandas
x_train = pd.DataFrame(x_train.reshape(len(x_train), 784))
x_test = pd.DataFrame(x_test.reshape(len(x_test), 784))
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)


#### Develop Model

think of a^L as the value inside a neuron
a^L = σ_1(w^L a^(L - 1) + b^L)

the cost function for a given training example is
(a^L - y)^2

 
  

In [213]:
class Neuron:
    def __init__(self, is_input_layer):
        if(is_input_layer==False):
            self.w = []
        self.value = 0

class Layer:
    def __init__(self, n_neurons, is_input_layer):
        self.b = np.random.rand()
        self.neurons = []
        for i in range(n_neurons):
            self.neurons.append(Neuron(is_input_layer))
        

In [218]:
class VisNet:
    
    def __init__(self, structure, epochs=10, alpha=0.1, batch_size=0, regularizer=0.0):
        
        self.size = [x for x in structure if isinstance(x, int)]
        self.activation_functions = [x.lower() for x in structure if isinstance(x, str)]

        self.epochs = epochs
        self.alpha = alpha
        self.batch_size = batch_size
        self.regularizer = regularizer

        self.input_size = self.size[0]
        self.output_size = self.size[-1]

        self.layers = [Layer(self.size[0], is_input_layer=True)]
        for i in range(1,len(self.size)):
            self.layers.append(Layer(self.size[i], is_input_layer=False))
            for Neuron in self.layers[i].neurons:
                for j in range(self.size[i-1]):
                    Neuron.w.append(np.random.rand())                

        
    def relu(self, x):
        return(max(0,x))

    def sigmoid(self, x):
        return(1/(1+np.exp(-x)))

    def softmax(self, x):
        for i in range(len(x)):
            x[i] = np.exp(x[i])
        return x/np.sum(x)

    def forward(self, x_sample):
        # feed input values into input layer
        for i in range(len(x_sample)):
            self.layers[0].neurons[i].value = x_sample[i]

        # for each layer (after the input layer)
        for k in range(1,len(self.size)):
            # for each neuron in the layer
            for i in range(len(self.layers[k].neurons)):
                # add the bias term from the previous layer
                self.layers[k].neurons[i].value = self.layers[k-1].b
                # for each neuron in the previous layer 
                for j in range(len(self.layers[k-1].neurons)):
                    # add the input * the weight
                    self.layers[k].neurons[i].value = self.layers[k].neurons[i].value + (self.layers[k-1].neurons[j].value*self.layers[k].neurons[i].w[j])
                # appply the activation function
                if(self.activation_functions[k-1] == 'relu'):
                    self.layers[k].neurons[i].value = self.relu(self.layers[k].neurons[i].value)
                if(self.activation_functions[k-1] == 'sigmoid'):
                    self.layers[k].neurons[i].value = self.sigmoid(self.layers[k].neurons[i].value)


    # IMPLEMENT SOFTMAX NEXT
    


#### Testing

In [232]:
###############################################################
clf = VisNet([784, "relu", 200, 'sigmoid', 80, 'softmax', 10])
###############################################################

assert(clf.relu(12))==12
assert(clf.relu(0))==0
assert(clf.relu(-3))==0
assert(clf.relu(0.02310320120))>0

assert(clf.sigmoid(0))==0.5
assert(clf.sigmoid(1))>0.5
assert(clf.sigmoid(1000))==1
assert(clf.sigmoid(-1000))==0

a,b,c = clf.softmax([1,4,5])
assert(a<b<c)
a,b,c,d = clf.softmax([-5,10,25,3])
assert((a<b)&(b>d))

assert(len(clf.layers[1].neurons[0].w))==784
assert(len(clf.layers[1].neurons[12].w))==784
assert(len(clf.layers[2].neurons[0].w))==200
assert(len(clf.layers[3].neurons[0].w))==80

print(clf.layers[3].neurons[0].value)
clf.forward(x_train.iloc[0])
print(clf.layers[3].neurons[0].value)

0
38.9167570581126


#### Final

In [ ]:
# # initialise the neural network
clf = VisNet([784, "relu", 200, 'sigmoid', 80, 'softmax', 10])
# # train the neural network
clf.train(x_train, y_train)
# # make predicitons using the neural network
# clf.predict(x_test)